# Lab 12: Robust Optimization 
In this lab, we will see some applications of robust optimization, namely a modified version of the Knapsack 0/1 problem, and the portfolio optimization problem.

Your job in this lab is to implement the missing functions, and study how different functions lead to different outcomes from both the point of view of the objective value and the probability of violating the constraints of the problem.

The examples are taken from https://xiongpengnus.github.io/rsome/ro_rsome, using the RSOME library for robust optimization.

In [ ]:
from rsome import ro
from rsome import grb_solver

import itertools as it
import rsome as rs
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt

In [ ]:
def norms(ps):
    d = np.linspace(-1.1, 1.1, 1000)
    xx, yy = np.meshgrid(d, d)
    l = 3
    fig, axs = plt.subplots(1, len(ps), sharey=True, figsize=(len(ps) * l, l))
    for (p, ax) in zip(ps, axs):
        z = np.array([
            [
                np.linalg.norm([xi, yi], p)
                for (xi, yi)
                in zip(x, y)
            ]
            for (x, y)
            in zip(xx, yy)
        ])
        cs = ax.contour(xx, yy, z, levels=np.linspace(-1, 1, 11))
        ax.clabel(cs, inline=True, fontsize=10)
        ax.set_xlim([-1.1, 1.1])
        ax.set_ylim([-1.1, 1.1])
        ax.set_title(f'norm {p}')

    fig.subplots_adjust(wspace=0, hspace=0)
    fig.savefig('out/norms.svg')


norms([1, 2, np.infty])

# The Knapsack 0/1 Problem
In this exercise, we will solve the Knapsack problem (seen in the previous labs), slightly modified in order to have uncertainties about the volumes of the items.

The uncertainty about the volumes is not the same for all the items. They are defined by $\delta$, defined as a fraction of the size of the volumes of the items.

In this exercise, you are asked to implement the definition of the uncertainty set in order to have both an **ellipsoidal** uncertainty set and a **finite** uncertainty set.

Try to implement different sizes for the ellipsoid and different interval for the finite set and compare the objective values and the probability of violating the constraints with the different setups.

In [ ]:
items = [
    {'name': 'apple', 'value': 1, 'volume': 2},
    {'name': 'pear', 'value': 2, 'volume': 2},
    {'name': 'banana', 'value': 2, 'volume': 2},
    {'name': 'watermelon', 'value': 5, 'volume': 10},
    {'name': 'orange', 'value': 3, 'volume': 2},
    {'name': 'avocado', 'value': 3, 'volume': 2},
    {'name': 'blueberry', 'value': 3, 'volume': 1},
    {'name': 'coconut', 'value': 4, 'volume': 3},
    {'name': 'cherry', 'value': 2, 'volume': 1},
    {'name': 'apricot', 'value': 1, 'volume': 1},
]

items.sort(key=lambda x: x['value'])

N = len(items)
b = 10  # budget

c = np.array([i['value'] for i in items]).flatten()  # profit coefficients
w = np.array([i['volume'] for i in items]).flatten()  # profit coefficients

delta = 0.2 * w  # maximum deviations

In [ ]:
fig, ax = plt.subplots()
ax.yaxis.tick_right()
ax.set_yticks(range(len(items)))
ax.set_yticklabels([f"{i['value']} : {i['name']}" for i in items])
ax.errorbar(w, range(len(items)), xerr=delta, capsize=5, fmt='.k')
fig.savefig('out/knapsack.svg')

In [ ]:
def robust(uncertainty_set):
    """
    The function robust implements the robust optimization model,
    given the budget of uncertainty r
    """

    model = rs.ro.Model('robust')
    # Boolean variable x (F: leave, T: keep)
    x = model.dvar(N, vtype='B')
    # Random variable
    z = model.rvar(N)

    # Uncertainty set
    z_set = uncertainty_set(z)
    # Maximize the value of the knapsack (i.e., the dot product between the values and x)
    model.max(c @ x)

    # Add constraint: the maximum (uncertain) volume is smaller than the budget
    model.st(((w + z * delta) @ x <= b).forall(z_set))

    # Solve
    model.solve(rs.grb_solver, display=False)

    # Return the optimal objective and solution
    return model.get(), x.get()


def sim(x_sol, zs):
    """
    The function sim is for calculating the probability of violation
    via simulations.
        x_sol: solution of the Knapsack problem
        zs: random sample of the random variable z
    """

    ws = w + zs * delta  # random samples of uncertain weights

    # did not respect the budget constraint
    # true / n
    return (ws @ x_sol > b).mean()

In [ ]:
# z is the U in the slide
# z * delta uncertain area

ellipsoidal = lambda z: (rs.norm(z, 2) <= 1.25)

finite_set = lambda z: (z == 0)

infinite_set = lambda z: (z <= 0.5)

In [ ]:
num_samples = 20000
# Generate random samples for z
zs = np.random.uniform(-1, 1, (num_samples, N))

objective_value, solution = robust(infinite_set)
prob_violation = sim(solution, zs)

print(dict(zip([item['name'] for item in items], [x > 0 for x in solution])))
print(f'TOT: {objective_value}')
print(f'P of violation: {prob_violation}')

# Robust Portfolio Optimization
In this problem, we want to build a portfolio (e.g., of stocks), by using robust approaches.

To be more specific, in this problem we have a set of fictionary stocks, each of which has different means and deviations for the returns.

Your job here is to implement a **box** uncertainty set to robustly optimize the portfolio.
Try different values for the box in order to study how the uncertainty affects the objective value of and the number of different stocks chosen.

In [ ]:
n = 10  # number of stocks

stocks = {
    f'Company {chr(65 + i)}': {'Mean': np.around(np.random.uniform(0.9, 1.1), 2),
                               'Deviation': np.around(np.random.uniform(0.1, 0.3), 2)}
    for i in range(n)
}

In [ ]:
def portfolio_optimization(uncertainty_set):
    p = np.array([stocks[s]['Mean'] for s in stocks])  # mean returns
    delta = np.array([stocks[s]['Deviation'] for s in stocks])  # deviations of returns

    model = rs.ro.Model()
    x = model.dvar(n)  # fractions of investment
    z = model.rvar(n)  # random variables

    z_set = uncertainty_set(z)

    model.maxmin(
        (p + delta * z) @ x,  # the max-min objective
        z_set
    )

    model.st(sum(x) == 1)  # summation of x is one
    model.st(x >= 0)  # x is non-negative

    model.solve(rs.grb_solver)  # solve the model by Gurobi
    return model.get(), x.get()

In [ ]:
us = [('Z2', lambda z: (rs.norm(z, 1) <= 2, rs.norm(z, np.infty) <= 1))
    , ('Z2L', lambda z: (rs.norm(z, 1) <= 2, rs.norm(z, np.infty) <= 1, z <= 0))
    , ('Z2G', lambda z: (rs.norm(z, 1) <= 2, rs.norm(z, np.infty) <= 1, z >= -0.5))
    , ('Z5', lambda z: (rs.norm(z, 1) <= 5, rs.norm(z, np.infty) <= 1))
    , ('Z5L', lambda z: (rs.norm(z, 1) <= 5, rs.norm(z, np.infty) <= 1, z <= 0))
    , ('Z5G', lambda z: (rs.norm(z, 1) <= 5, rs.norm(z, np.infty) <= 1, z >= -0.5))
    , ('Z8', lambda z: (rs.norm(z, 1) <= 8, rs.norm(z, np.infty) <= 1))
    , ('Z8L', lambda z: (rs.norm(z, 1) <= 8, rs.norm(z, np.infty) <= 1, z <= 0))
    , ('Z8G', lambda z: (rs.norm(z, 1) <= 8, rs.norm(z, np.infty) <= 1, z >= -0.5))
      ]

In [ ]:
layout = [
    [0, 0, 0]
    , [1, 2, 3]
    , [4, 5, 6]
    , [7, 8, 9]
]

fig, axs = plt.subplot_mosaic(layout, figsize=(5 * 3, 5 * 4))

ax = axs[0]
ax.yaxis.tick_right()
ax.set_yticks(range(len(items)))
ax.set_yticklabels([s.split(' ')[1] for s in stocks])
ax.errorbar([stocks[s]['Mean'] for s in stocks], range(len(stocks)), xerr=[stocks[s]['Deviation'] for s in stocks],
            capsize=5, fmt='.k')
ax.tick_params(top=False, bottom=True, left=False, right=False, labelleft=False, labelbottom=True)

for i, x in enumerate([stocks[s] for s in stocks]):
    ax.annotate(f'{x["Mean"]} \u00B1 {x["Deviation"]}', (x['Mean'], i), textcoords="offset points", xytext=(0, 8))

ax.invert_yaxis()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

for i, (name, u) in enumerate(us):
    bx = axs[i]
    obj_val, x_sol = portfolio_optimization(u)
    print('Objective value: {0:0.4f}'.format(obj_val))

    recipe = [s.split(' ')[1] + ' : ' + '{:.0f}'.format(x * 100) for s, x in zip(stocks, x_sol)]
    data = [x for x in x_sol]

    wedges, texts = bx.pie(data, wedgeprops=dict(width=0.5), startangle=-40)

    bx.set_title(name)
    bx.text(0, 0, '{:.2f}'.format(obj_val), ha='center', va='center')
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"), bbox=bbox_props, zorder=0, va="center")

    for (i, p), f in zip(enumerate(wedges), x_sol):
        ang = (p.theta2 - p.theta1) / 2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        bx.annotate(
            recipe[i],
            xy=(x, y),
            xytext=(1.35 * np.sign(x), 1.4 * y),
            horizontalalignment=horizontalalignment,
            **kw
        )

fig.subplots_adjust(wspace=0.5)
fig.savefig(f'out/portfolio.svg')